# Desta vez 

In [17]:
import os
import numpy as np
from typing import List, Tuple, Set
from scipy.spatial import ConvexHull as ScipyHull

EPSILON = 1e-9

class Face:
    def __init__(self, i: int, j: int, k: int, points: np.ndarray):
        """Inicializa uma face triangular com três índices de pontos."""
        self.indices = (i, j, k)
        self.points = points
        self.outside_set: Set[int] = set()
        self.compute_plane()

    def compute_plane(self):
        """Calcula o plano da face a partir dos três pontos e salva a normal e o deslocamento."""
        p0, p1, p2 = (self.points[idx] for idx in self.indices)
        u = p1 - p0
        v = p2 - p0
        n = np.cross(u, v)
        norm = np.linalg.norm(n)
        if norm < 1e-9:
            raise ValueError("Degenerate face")
        self.normal = n / norm
        self.offset = np.dot(self.normal, p0)

    def distance(self, p: np.ndarray) -> float:
        """Retorna a distância do ponto p ao plano da face."""
        return np.dot(self.normal, p) - self.offset

    def orient_towards(self, interior_point: np.ndarray):
        """Garante que a normal da face aponte para fora do casco (longe do ponto interior)."""
        if self.distance(interior_point) > 0:
            i, j, k = self.indices
            self.indices = (i, k, j)
            self.compute_plane()

class IncrementalConvexHull3D:
    def __init__(self, points: List[Tuple[float, float, float]], export_dir: str = None):
        """Inicializa o algoritmo com os pontos de entrada e diretório opcional para exportação VTK."""
        self.points = np.array(points)
        self.faces: List[Face] = []
        self.export_dir = export_dir
        if self.export_dir:
            os.makedirs(self.export_dir, exist_ok=True)
        self.step = 0

    def compute(self):
        """Executa o algoritmo incremental para construir o casco convexo e exporta os passos."""
        idx = self._build_initial_tetrahedron()
        interior = np.mean(self.points[idx], axis=0)
        combos = [(idx[0], idx[1], idx[2]),
                  (idx[0], idx[1], idx[3]),
                  (idx[0], idx[2], idx[3]),
                  (idx[1], idx[2], idx[3])]
        for i, j, k in combos:
            f = Face(i, j, k, self.points)
            f.orient_towards(interior)
            self.faces.append(f)

        for pi in range(len(self.points)):
            if pi in idx:
                continue
            for face in self.faces:
                if face.distance(self.points[pi]) > EPSILON:
                    face.outside_set.add(pi)

        self._export_step("initial")

        while True:
            # Escolhe a face com maior conjunto externo
            face = max(self.faces, key=lambda f: len(f.outside_set), default=None)
            if face is None or not face.outside_set:
                break
            far_idx = max(face.outside_set, key=lambda pi: face.distance(self.points[pi]))
            far_pt = self.points[far_idx]
            visible = [f for f in self.faces if f.distance(far_pt) > EPSILON]

            # Encontra a borda do horizonte
            edge_count = {}
            for f in visible:
                i, j, k = f.indices
                for edge in [(i,j), (j,k), (k,i)]:
                    key = tuple(sorted(edge))
                    edge_count[key] = edge_count.get(key, 0) + 1

            horizon = [e for e, c in edge_count.items() if c == 1]
            self.faces = [f for f in self.faces if f not in visible]

            new_faces = []
            for i, j in horizon:
                f = Face(i, j, far_idx, self.points)
                f.orient_towards(interior)
                new_faces.append(f)

            all_out = set().union(*(f.outside_set for f in visible))
            for f in new_faces:
                f.outside_set = set()
            for pi in all_out:
                if pi == far_idx:
                    continue
                for f in new_faces:
                    if f.distance(self.points[pi]) > EPSILON:
                        f.outside_set.add(pi)
            self.faces.extend(new_faces)

            self.step += 1
            self._export_step(f"step_{self.step:03d}")

        return [face.indices for face in self.faces]

    def _build_initial_tetrahedron(self) -> List[int]:
        """Procura quatro pontos não coplanares para formar o primeiro tetraedro."""
        n = len(self.points)
        for i in range(n - 3):
            for j in range(i + 1, n - 2):
                for k in range(j + 1, n - 1):
                    for l in range(k + 1, n):
                        tetra = self.points[[i, j, k, l]]
                        vol = np.abs(np.linalg.det(np.vstack((tetra[1] - tetra[0],
                                                              tetra[2] - tetra[0],
                                                              tetra[3] - tetra[0]))))
                        if vol > 1e-6:
                            return [i, j, k, l]
        raise ValueError("All points are coplanar")

    def _export_step(self, label: str):
        """Salva o casco atual em formato VTK com o nome hull_<label>.vtk."""
        if not self.export_dir:
            return
        filepath = os.path.join(self.export_dir, f"hull_{label}.vtk")
        all_indices = sorted({idx for face in self.faces for idx in face.indices})
        idx_map = {old: new for new, old in enumerate(all_indices)}
        with open(filepath, "w", encoding="utf-8") as f:
            f.write("# vtk DataFile Version 3.0\nIncremental Convex Hull 3D\nASCII\nDATASET POLYDATA\n")
            f.write(f"POINTS {len(all_indices)} float\n")
            for i in all_indices:
                f.write("{} {} {}\n".format(*self.points[i]))
            f.write(f"POLYGONS {len(self.faces)} {len(self.faces) * 4}\n")
            for face in self.faces:
                a, b, c = (idx_map[i] for i in face.indices)
                f.write(f"3 {a} {b} {c}\n")

def validate_against_scipy(points: List[Tuple[float, float, float]], faces: List[Tuple[int, int, int]]):
    """Valida os resultados comparando com o ConvexHull do SciPy."""
    scipy_hull = ScipyHull(points)
    scipy_faces = {tuple(sorted(f)) for f in scipy_hull.simplices}
    our_faces = {tuple(sorted(f)) for f in faces}

    if scipy_faces == our_faces:
        print("✅ Convex hull matches SciPy implementation.")
    else:
        missing = scipy_faces - our_faces
        extra = our_faces - scipy_faces
        print("❌ Mismatch with SciPy convex hull.")
        print(f"Faces missing from our hull: {len(missing)}")
        print(f"Extra faces in our hull: {len(extra)}")

Pode mudar a variável pts abaixo para incluir seus pontos ou usar a função rand_sphere como vista abaixo. Validei a resposta contra uma biblioteca de solução da ScikitLearn para garantir que o resultado está correto.

In [21]:
pts = [rand_sphere() for _ in range(70)]


if __name__ == "__main__":
    def rand_sphere():
        while True:
            p = np.random.uniform(-1, 1, 3)
            if np.linalg.norm(p) <= 1:
                return tuple(p)
    sim_folder = "vtk_simulation"
    solver = IncrementalConvexHull3D(pts, export_dir=sim_folder)
    face_indices = solver.compute()
    validate_against_scipy(pts, face_indices)
    print(f"✅ Simulation steps written to {sim_folder}/")


✅ Convex hull matches SciPy implementation.
✅ Simulation steps written to vtk_simulation/
